In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [39]:
df=pd.read_csv("/kaggle/input/european-restaurant-reviews/European Restaurant Reviews.csv")

In [40]:
df

,Country,Restaurant Name,Sentiment,Review Title,Review Date,Review
0,France,The Frog at Bercy Village,Negative,Rude manager,May 2024 •,The manager became agressive when I said the c...
1,France,The Frog at Bercy Village,Negative,A big disappointment,Feb 2024 •,"I ordered a beef fillet ask to be done medium,..."
2,France,The Frog at Bercy Village,Negative,Pretty Place with Bland Food,Nov 2023 •,"This is an attractive venue with welcoming, al..."
3,France,The Frog at Bercy Village,Negative,Great service and wine but inedible food,Mar 2023 •,Sadly I used the high TripAdvisor rating too ...
4,France,The Frog at Bercy Village,Negative,Avoid- Worst meal in Rome - possibly ever,Nov 2022 •,From the start this meal was bad- especially g...
...,...,...,...,...,...,...
1497,Cuba,Old Square (Plaza Vieja),Negative,The Tourism Trap,Oct 2016 •,Despite the other reviews saying that this is ...
1498,Cuba,Old Square (Plaza Vieja),Negative,the beer factory,Oct 2016 •,beer is good. food is awfull The only decent...
1499,Cuba,Old Square (Plaza Vieja),Negative,brewery,Oct 2016 •,"for terrible service of a truly comedic level,..."
1500,Cuba,Old Square (Plaza Vieja),Negative,It's nothing exciting over there,Oct 2016 •,We visited the Havana's Club Museum which is l...


In [41]:
df["Restaurant Name"].unique()

array(['The Frog at Bercy Village',
       'Ad Hoc Ristorante (Piazza del Popolo)', 'Stara Kamienica',
       'Mosaic', 'Pelmenya', 'The LOFT', 'Old Square (Plaza Vieja)'],
      dtype=object)

# # text preprocessing

In [42]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'\W+', ' ', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    
    # Join tokens back into a string
    return ' '.join(tokens)

# df["Pre_Review"]=df["Review"].apply(prep)
for col in ['Review Title','Review']:
#     df[col] = df[col].apply(lambda row: preprocess_text("".join(row.values.astype('str'))),axis=1)
      df[col]=df[col].apply(preprocess_text)
# preprocessed_description = preprocess_text(description)

# print(preprocessed_headline)
# print(preprocessed_description)
df

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Country,Restaurant Name,Sentiment,Review Title,Review Date,Review
0,France,The Frog at Bercy Village,Negative,rude manager,May 2024 •,manager became agressive said carbonara good r...
1,France,The Frog at Bercy Village,Negative,big disappointment,Feb 2024 •,ordered beef fillet ask done medium got well d...
2,France,The Frog at Bercy Village,Negative,pretty place bland food,Nov 2023 •,attractive venue welcoming albeit somewhat slo...
3,France,The Frog at Bercy Village,Negative,great service wine inedible food,Mar 2023 •,sadly used high tripadvisor rating literally f...
4,France,The Frog at Bercy Village,Negative,avoid worst meal rome possibly ever,Nov 2022 •,start meal bad especially given price visited ...
...,...,...,...,...,...,...
1497,Cuba,Old Square (Plaza Vieja),Negative,tourism trap,Oct 2016 •,despite reviews saying lovely place hang espec...
1498,Cuba,Old Square (Plaza Vieja),Negative,beer factory,Oct 2016 •,beer good food awfull decent thing shish kabob...
1499,Cuba,Old Square (Plaza Vieja),Negative,brewery,Oct 2016 •,terrible service truly comedic level 2 3 full ...
1500,Cuba,Old Square (Plaza Vieja),Negative,nothing exciting,Oct 2016 •,visited havana club museum located old havana ...


# # Vectorizer

In [43]:
vect_tf=[]
feature_names=[]
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine the headlines and descriptions for vectorization
for col in ["Review Title","Review"]:
    test_data = df[col].astype('str').tolist()

# Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

# Fit and transform the text data
    X = vectorizer.fit_transform(test_data)
    vect_tf.append(X.toarray())
    feature_names.append(vectorizer.get_feature_names_out())

    
print(X.shape)


(1502, 6134)


In [44]:
mf=[]
for i,mat in enumerate(vect_tf):
    print(mat.shape)
    mf.append(pd.DataFrame(mat,columns=feature_names[i]))
# mf

(1502, 1041)
(1502, 6134)


In [45]:
d=np.concatenate(mf,axis=1)
dfd=pd.DataFrame(d)
dfd

,0,1,2,3,4,5,6,7,8,9,...,7165,7166,7167,7168,7169,7170,7171,7172,7173,7174
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["Country","Restaurant Name"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())
one.columns=[i for i,x in enumerate(one.columns)]

In [53]:
one

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1499,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [54]:
df["Sentiment"].nunique()

2

In [55]:
concaty=pd.concat([dfd,one],axis=1)


In [56]:
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(df["Sentiment"])

In [57]:
X_train,X_test,Y_train,Y_test=train_test_split(concaty,Y,test_size=0.2,random_state=42)

In [61]:
random_classif=RandomForestClassifier()
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

0.9269102990033222

In [64]:
grad_classif=GradientBoostingClassifier()
grad_classif.fit(X_train,Y_train)
grad_classif.score(X_test,Y_test)

0.9169435215946844

In [65]:
logistic_classif=LogisticRegression()
logistic_classif.fit(X_train,Y_train)
logistic_classif.score(X_test,Y_test)

0.946843853820598

In [66]:
ada_classif=AdaBoostClassifier()
ada_classif.fit(X_train,Y_train)
ada_classif.score(X_test,Y_test)

0.9102990033222591

In [67]:
svm_classif=SVC()
svm_classif.fit(X_train,Y_train)
svm_classif.score(X_test,Y_test)

0.9501661129568106

In [68]:
deci_classif=DecisionTreeClassifier()
deci_classif.fit(X_train,Y_train)
deci_classif.score(X_test,Y_test)

0.9136212624584718